<a href="https://colab.research.google.com/github/matheus-de-araujo/fakenews-classifier-notebook/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Importação das bibliotecas

In [1]:
import pandas
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
import pickle
from sklearn.svm import SVC
import os, re
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/matheus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 2. Leitura do dataset

In [12]:
if os.path.exists('dataset-fake-br-for-tcc/'):
    print('A pasta já existe!')
else:
    !git clone https://github.com/matheus-de-araujo/dataset-fake-br-for-tcc.git

def returnDataSetFrame():
    dataFrame = pandas.DataFrame(columns=[ 'text', 'label'])
    paths = ['dataset-fake-br-for-tcc/full_texts/fake/', 'dataset-fake-br-for-tcc/full_texts/true/']

    for filename in os.listdir(paths[0]):
        with open(paths[0]+filename, 'r') as f:
            text = f.read()
            dataFrame.loc[len(dataFrame)] = [preProcessing(text), 'fake']

    for filename in os.listdir(paths[1]):
        with open(paths[1]+filename, 'r') as f:
            text = f.read()
            dataFrame.loc[len(dataFrame)] = [preProcessing(text), 'true']
    return dataFrame

data = returnDataSetFrame()

A pasta já existe!


# 3. Limpeza dos dados e Remoção dos StopWords

In [3]:
nltk.download('stopwords')
nltk.download('rslp')

def cleaning_text(text):
    text = text.lower()
    text = re.sub("(\\d|\\W|\d+\w+|\\n)+"," ",text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

def removeStopWords(text):
    stopwords_nltk = nltk.corpus.stopwords.words('portuguese')
    text = ' '.join([word for word in text.split() if word not in stopwords_nltk])
    return text

def preProcessing(text):
    return removeStopWords(cleaning_text(text))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/matheus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /home/matheus/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [ ]:
data

In [ ]:
y = data.label
X_train, X_test, y_train, y_test = train_test_split(data['text'], y, test_size=0.2, random_state=42)

In [ ]:
tf_vectorizer = TfidfVectorizer(stop_words=stopwords.words('portuguese'), analyzer='word', ngram_range=(1, 1), lowercase=True, use_idf=True)

In [ ]:
tfidf_train = tf_vectorizer.fit_transform(X_train) 

In [ ]:
def new_func():
    tfidf_test = tf_vectorizer.transform(X_test)

new_func()

In [ ]:
pickle.dump(tf_vectorizer,open('tfidf.pkl','wb')) 

In [ ]:
support_vector_machine = SVC(kernel='linear').fit(tfidf_train, y_train)

In [ ]:
pickle.dump(support_vector_machine, open('model.pkl','wb'))

In [ ]:
model = pickle.load(open('model.pkl','rb'))